In [ ]:
import numpy as np
from conv import Convolutional_code
from mod import Modulation
from channel import AWGN
import os
import os.path
from tqdm import tqdm

In [ ]:
# 데이터 셋 폴더 생성 위치
dest = r"C:\Users\USER-PC\haneol\DATASET_I" 

# SNR 범위 설정 : 1
SNR = range(-20, 21, 1)

if not os.path.exists(dest):
    os.mkdir(dest)
    print("데이터 셋 폴더 생성")

for CL in range(3):
    # 채널 코딩에 사용할 콘볼루션 코드 선택 (1, 2, or 3)
    convolution_code = CL + 1
    modulation = 1
    
    encoder = Convolutional_code(convolution_code)  
    mod = Modulation(modulation)
    
    for i in tqdm(SNR, desc="Loading Files"):
        # 파일 생성 폴더 지정
        dest_dir = os.path.join(dest, str(i))
        
        if not os.path.exists(dest_dir):
            os.mkdir(dest_dir)
            print(str(i) + " 폴더 생성")
        
        for j in range (1, 501, 1):
            # 난수 비트 생성 : 결과 값 128비트로 맞추기
            if CL == 0:
                SIZE = 62
            elif CL == 1:
                SIZE = 61
            else:
                SIZE = 60
            random_bits = np.random.randint(2, size=SIZE)
            
            # 채널코딩
            encoded, _ = encoder.encoder(list(random_bits))  

            # 변조
            moded = mod.mod(encoded)  
            
            # 채널 효과 적용
            noise_data = AWGN(int(i), 1/2, 1, moded)
            
            # 복조
            data = mod.demod(noise_data, 1)
            
            # 넘파이 배열로 변경
            data = np.array(data).reshape(128, 1)
            
            # 파일명 설정
            name = "CONV_" + str(CL+3) + "_" + str(j) + ".npy"
            
            # npy 파일로 저장
            np.save(os.path.join(dest_dir, name), data)
    
    print("CONV 2_1_" + str(CL+3) + " 생성 완료")

print("생성 완료")

In [ ]:
# (23, 12) Golay code
m_len = 12

P = np.array([
    [1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1],
    [1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1],
    [1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1],
    [1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0],
    [0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0],
    [0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1],
    [1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0],
    [0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0],
    [0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1],
    [1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0],
    [0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1],
], dtype='int')

G = np.concatenate((np.eye(m_len, dtype="int"), P), axis=1)

msg = np.random.randint(2, size=(1, m_len))
codeword = np.dot(msg, G) %2

In [ ]:
print(msg)
print(codeword)
print((-1)**codeword)
send_code = (-1)**codeword
noise_power = 1
noise = noise_power*np.random.normal(size = 23)
received_vector = send_code + noise
print()

In [ ]:
# R(r, m) RM code, r=3까지는 가능, 4부터는 안됨

def G_matrix(length, m, r):
    G = np.ones(length)
    for i in range(m):
        v = np.zeros((int(length/(2**(i+1)))))
        v = np.hstack((v, np.ones((int(length/(2**(i+1)))))))
        while v.shape[0] < length :
            v = np.hstack((v, np.zeros((int(length/(2**(i+1)))))))
            v = np.hstack((v, np.ones((int(length/(2**(i+1)))))))
        G = np.vstack((G,v))
    if r == 1:
        return G
    elif r > 1 :
        for i in range(1,m):
            for j in range(i+1,m+1):
                G = np.vstack((G,(G[i]*G[j])))
        if r == 3:
            G = np.vstack((G,(G[1]*G[2]*G[3])))
            G = np.vstack((G,(G[1]*G[3]*G[4])))
            G = np.vstack((G,(G[1]*G[2]*G[4])))
            G = np.vstack((G,(G[2]*G[3]*G[4])))
        return G
    return G

m = 4
r = 1
length = 2**m

if r == 1:
    masking_length=0
    msg_length = m+r
elif r == 2:
    masking_length=6
    msg_length = 11
elif r == 3:
    masking_length=10
    msg_length = 15

G = G_matrix(length, m, r)
msg = np.random.randint(2,(1,msg_length))
codeword = np.dot(msg, G) %2

In [ ]:
# BCH code

c_length = 15
m_length = 7

# BCH 7,4
# G(x) = x^3 + x^2 + 1
G_X_7_4 =[1,1,0,1]

# BCH 15,7
# G(x) = x^8+x^7+x^6+x^4+1
G_X_15_7 = [1,1,1,0,1,0,0,0,1]

BCH_7_4_G = np.zeros((4,7))

for i in range(4):
    BCH_7_4_G[i,i:i+4] = G_X_7_4

for i in range(4):
    for j in range(i+1,4):
        if BCH_7_4_G[i, j] == 1:
            BCH_7_4_G[i] = (BCH_7_4_G[i] + BCH_7_4_G[j])%2

BCH_15_7_G = np.zeros((7,15))

for i in range(7):
    BCH_15_7_G[i,i:i+9] = G_X_15_7

for i in range(7):
    for j in range(i+1,7):
        if BCH_15_7_G[i, j] == 1:
            BCH_15_7_G[i] = (BCH_15_7_G[i] + BCH_15_7_G[j])%2

G = BCH_15_7_G # G =BCH_7_4_G
msg = np.random.randint(2,(1,msg_length))
codeword = np.dot(msg, G)%2

In [ ]:
# (n, k) Polar code

def make_F(power):
    F = np.array([[1,0],[1,1]])
    for i in range(1,power):
        first = np.concatenate((F,np.zeros((2**i, 2**i))), axis=1)
        second = np.concatenate((F,F), axis=1)
        F = np.concatenate((first, second), axis =0)

    return F
def Compute_z(z, k, i = 1):
    for j in range(i):
        z[(2*i,2*j)] = 2*z[(i,j)] - (z[(i,j)])**2
        z[(2*i,2*j+1)] = (z[(i,j)])**2
    if 2*i < 2**k:
        z = Compute_z(z, k, 2*i)

    return z

def Frozen_bits(n, z, slice_index):
    bit_index = np.zeros(n)
    for i in range(n):
        bit_index[i] = z[(n, i)]

    bit_index = np.argsort(bit_index)[::-1]

    frozen_bit_index = bit_index[:slice_index]
    message_bit_index = bit_index[slice_index:]
    
    return np.sort(frozen_bit_index), np.sort(message_bit_index)

k = 8
n = 2*k
power = int(np.log2(n))

F = make_F(power)

z = {}
z[(1,0)] = 0.5
z = Compute_z(z, power)

frozen_bit_index, message_bit_index = Frozen_bits(n, z, int(n-k))

msg = np.random.randint(0,2,(k))
u = np.zeros((n))
u[message_bit_index] = msg

codeword = np.dot(u,F)%2